In [3]:
import numpy as np
import pandas as pd
import itertools

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set_style('darkgrid')
sns.set_context('talk', font_scale=1.5)
sns.set(color_codes=True)

%matplotlib inline

def lighten_color(color, degree):
    cin = matplotlib.colors.colorConverter.to_rgb(color)
    cw = np.array([1.0]*3)
    return tuple(cin + (cw - cin)*degree)

In [4]:
from succinogenes.coll_setup import (setup_collocation, initialize_collocation,
                                     calc_max_productivity, calc_max_yield)

succ_coll = setup_collocation([4, 4, 4])
succ_max_prod = calc_max_productivity(succ_coll)

coll = succ_coll


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



In [5]:
ts = np.linspace(0, coll.var.tf_op, 100)
sol = coll._interpolate_solution(ts)
deriv = coll._interpolate_derivative(ts)

fstage_starts = np.hstack([np.array(0.),
                           np.cumsum(np.array(coll.stage_breakdown) *
                                     coll.var.h_op)])

fstage_ends = np.hstack([np.cumsum(np.array(coll.stage_breakdown) *
                                   coll.var.h_op)])

from scipy.interpolate import lagrange

const_array = coll.var.a_op

a_ts = np.empty(len(ts))

stage_starts = [0.]
stage_ends = [coll.var.h_op[0]]
for i in range(coll.nk-1):
    stage_starts += [coll.var.h_op[coll._get_stage_index(i)] +
                     stage_starts[-1]]
    stage_ends += [coll.var.h_op[coll._get_stage_index(i)] +
                     stage_starts[-1]]

stage_starts = pd.Series(stage_starts)
stages = stage_starts.searchsorted(ts, side='right') - 1

for ki in range(coll.nk):
    interp = lagrange(coll.col_vars['tau_root'][1:], 
                      const_array[ki])

    a_ts[stages == ki] = interp(
        (ts[stages == ki] - stage_starts[ki]) /
        coll.var.h_op[coll._get_stage_index(ki)])
        
t_f = coll.col_vars['tgrid'][:, 1:].flatten()
a_f = coll.var.a_op.flatten()

In [ ]:
tg_f = coll.col_vars['tgrid'].flatten()
sol_f = coll.var.x_op.reshape((len(tg_f), coll.nx))

In [ ]:
sol[:,1].max()

In [ ]:
with sns.axes_style('ticks'):
    fig, axmatrix = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8, 4))
    ax1 = axmatrix[0]
    ax2 = axmatrix[1]
    ax_biomass = ax1.twinx()
    
    conc_lines = ax1.plot(ts, sol[:,-5:-1], ls=(5, (3, 3)), lw=1)
    conc_points = ax1.plot(tg_f, sol_f[:,-5:-1], '.')
    
    for line, point in zip(conc_lines, conc_points):
        point.set_color(line.get_color())
    
    biomass_lines = ax_biomass.plot(ts, sol[:, 0],  color=sns.color_palette()[-2], ls=(5, (3, 3)), lw=1.)
    biomass_points = ax_biomass.plot(tg_f, sol_f[:, 0], '.', color=sns.color_palette()[-2])
    
    formate_line = ax_biomass.plot(ts, sol[:, -1],  color=sns.color_palette()[-1], ls=(5, (3, 3)), lw=1.)
    formate_point = ax_biomass.plot(tg_f, sol_f[:, -1], '.', color=sns.color_palette()[-1])
    
    
    ax2.plot(t_f, a_f, 'k.')
    line = ax2.plot(ts, a_ts, 'k', lw=1., ls=(5, (3, 3)))    
    
    
    
    ax_biomass.set_ylim([0, 5])
    
    colors = itertools.cycle(['w', 'k'])
    
    for ax in axmatrix:
        for start, end, color in zip(stage_starts, stage_ends, colors):
            ax.axvspan(start, end, facecolor=lighten_color(color, 0.9), edgecolor='none')
            
        for end in fstage_ends[:-1]:
            ax.axvline(end, color='k')
        
        for t_grid in t_f:
            ax.axvline(t_grid, color=lighten_color(color, 0.5), lw=1., linestyle=':')
        
    ax1.set_xlim([0, coll.var.tf_op])
    
    ax1.legend(zip(conc_lines + biomass_lines + formate_line, 
                   conc_points + biomass_points + formate_point),
           ['ATP', 'Glucose', 'Succinate', 'Acetate', 'Biomass', 'Formate'],
          ncol=6, loc='upper center', bbox_to_anchor=(.5,-0.05))
    
    sns.despine(ax=ax1, right=False, top=True)
    sns.despine(ax=ax_biomass, right=False, top=True)
    sns.despine(ax=ax2, right=True)
    
    ax_biomass.set_ylabel('Biomass (g DCW/L)\nFormate (mM)')
    ax1.set_ylabel('Concentration (mM)')
    
    ax2.set_ylabel('EFM Activity\n(mmol/g DCW * hr)')
    
    for i, (start, end) in enumerate(zip(fstage_starts, fstage_ends)):
        ax2.text(start + (end-start)/2., 2.1, 'Stage {}'.format(i + 1), ha='center', va='bottom', fontdict={'size' : 12, 'weight' : 'bold'})
    
    ax2.set_xticks(t_f[:3].tolist(), minor=True)
    ax2.set_xticklabels([r'$\tau_1$', r'$\tau_2$', r'$\tau_3$'], minor=True)
    ax1.xaxis.set_tick_params(which='minor', bottom=False)
    plt.setp(ax1.get_xticklabels(which='minor'), visible=False)
    
    ax2.set_xlabel('Time (hr)')
    
    fig.subplots_adjust(hspace=0.3)
    
    fig.savefig('figs/coll_optimal.svg')

In [ ]:
def build_rxn_string(efm):
    efm = efm.copy()
    efm /= -efm[efm < -1E-6].sum()
    efm.index = efm.index.str.replace('_e', '').str.replace('EX_', '')
    reactants = efm[efm < -1E-2]
    products = efm[efm > 1E-2]
    reactant_bits = ' + '.join(['{:0.2f} {}'.format(-stoich, name) for
                                name, stoich in reactants.iteritems()])
    product_bits = ' + '.join(['{:0.2f} {}'.format(stoich, name) for
                               name, stoich in products.iteritems()])
    return reactant_bits + ' --> ' + product_bits


v_op = pd.DataFrame(coll.var.v_op,
                    columns=[build_rxn_string(efm)
                             for i, efm in coll.efms_float.iterrows()]).T
v_op = v_op[v_op.sum(1) > 1E-4]

In [ ]:
fig = plt.figure(figsize=[8, 4])
ax1 = fig.add_subplot(131, aspect='equal')
ax2 = fig.add_subplot(132, aspect='equal')
ax3 = fig.add_subplot(133, aspect='equal')

for i, ax in enumerate([ax1, ax2, ax3]):
    ax.set_title('Stage {}'.format(i+1))


    
pie1 = ax1.pie(v_op[0].values, startangle=90., colors=sns.color_palette("cubehelix", 5))
pie2 = ax2.pie(v_op[1].values, startangle=90., colors=sns.color_palette("cubehelix", 5))
pie3 = ax3.pie(v_op[2].values, startangle=90., colors=sns.color_palette("cubehelix", 5))

for pie_wedge in itertools.chain(pie1[0], pie2[0], pie3[0]):
    pie_wedge.set_edgecolor('white')
    pie_wedge.set_linewidth('2')
    
ax3.legend(loc=[1.05, .25], handles=pie3[0], labels=v_op.index.tolist())

fig.savefig('figs/efm_breakdown.svg')